# AnyDoor+Segment-anything

In [1]:
import numpy as np
import gradio as gr
from PIL import Image, ImageDraw
from omegaconf import OmegaConf
from run_inference import inference_single_image
from segment_anything import sam_model_registry, SamPredictor
sam_checkpoint = "/home/zjt/model/sam_vit_h_4b8939.pth"
sam = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)
sam.to("cuda")
predictor = SamPredictor(sam)
DConf = OmegaConf.load('./configs/datasets.yaml')

/home/zjt/anaconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logging improved.
ControlLDM: Running in eps-prediction mode
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is None and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 320, context_dim is 1024 and using 5 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is None and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 640, context_dim is 1024 and using 10 heads.
Setting up MemoryEfficientCrossAttention. Query dim is 1280, context_dim is None and using 20 heads.
Setting up MemoryEfficientCrossAttention. 

In [4]:

# 点击图片时画个点在上面
def on_click_image(image, global_state, evt: gr.SelectData):
    radius_scale= 0.01
    p_color = (255, 0, 0)
    xy = evt.index
    if global_state['img'] is None:
        global_state['img'] = image
    global_state['points'].append(xy)
    overlay_rgba = Image.new("RGBA", image.size, 0)
    overlay_draw = ImageDraw.Draw(overlay_rgba)
    rad_draw = int(image.size[0] * radius_scale)
    for p in global_state['points']:
        p_draw = int(p[0]), int(p[1])
        overlay_draw.ellipse(
            (
                p_draw[0] - rad_draw,
                p_draw[1] - rad_draw,
                p_draw[0] + rad_draw,
                p_draw[1] + rad_draw,
            ),
            fill=p_color,
        )
    image_draw =  Image.alpha_composite(image.convert("RGBA"), overlay_rgba).convert("RGB")
    return image_draw,global_state
# 推理时画上mask
def run_predict(img,global_state):
    image = np.array(global_state['img'])
    predictor.set_image(image)
    input_point = np.array(global_state['points'])
    input_label = np.array([1]*len(global_state['points']))
    if global_state['mask_input'] is None:
        masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )
        global_state['mask_input'] = logits[np.argmax(scores), :, :]
    else:
        masks, _, _ = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        mask_input=global_state['mask_input'][None, :, :],
        multimask_output=False,
    )
    global_state['mask'] = masks[0]
    im_mask = np.uint8(masks[0] * 255)
    im_mask = Image.fromarray(im_mask)
    image_draw = Image.blend(img, im_mask.convert('RGB'), 0.5)
    return image_draw,global_state
# 
def run2(gl1,gl2):
    ref_image = gl1['img']
    ref_image = np.array(ref_image)
    gt_image = gl2['img']
    gt_image = np.array(gt_image)
    ref_mask = gl1['mask'].astype(np.uint8)
    tar_mask = gl2['mask'].astype(np.uint8)
    gen_image = inference_single_image(ref_image, ref_mask, gt_image.copy(), tar_mask)
    out = Image.fromarray(gen_image)
    return out
   
with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            with gr.Column():
                global_state = gr.State({'img':None,'points':[],'mask_input':None,'mask':None})
                im = gr.Image(type='pil',value='/home/zjt/workspace/img2img/AnyDoor/examples/Gradio/FG/10036.jpg')
                
                im.select(
                    on_click_image,
                    inputs=[im, global_state],
                    outputs=[im,global_state],
                    queue=False,
                )
                btn = gr.Button()
                btn.click(fn=run_predict,inputs=[im,global_state],outputs=[im,global_state])
            with gr.Column():
                global_state2 = gr.State({'img':None,'points':[],'mask_input':None,'mask':None})
                im2 = gr.Image(type='pil',value='output.png')
                
                im2.select(
                    on_click_image,
                    inputs=[im2, global_state2],
                    outputs=[im2,global_state2],
                    queue=False,
                )
                btn2 = gr.Button()
                btn2.click(fn=run_predict,inputs=[im2,global_state2],outputs=[im2,global_state2])
    baseline_gallery = gr.Image()
    btn3 = gr.Button()
    btn3.click(fn=run2,inputs=[global_state,global_state2],outputs=[baseline_gallery])
demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps


DDIM Sampler: 100%|██████████| 50/50 [00:10<00:00,  5.00it/s]
